In [10]:
!pip install --upgrade pip
!pip install --upgrade keras
!pip uninstall tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.13.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.13.0


In [11]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5a/f2/5c2f878c62c8b79c629b11b33516bb55054d7677eba6f56f3a20296b56bd/tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)


In [12]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras.layers import LeakyReLU
#from keras.layers import relu
#from keras.layers import softnmax
#from keras.layers import Tahn
#from keras.layers import sign
from tensorflow.keras.optimizers import Adam
from keras.datasets import mnist
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = (x_test.astype(np.float32) - 127.5)/127.5
mnist_data=x_test.reshape(10000, 784)
print(mnist_data.shape)
len(mnist_data)

(10000, 784)


10000

In [14]:
def create_generator():
    generator = Sequential()
    generator.add(Dense(units=256,input_dim=100))
    generator.add(LeakyReLU(0.2))
    #generator.add(ReLU(0.2))
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    #generator.add(ReLU(0.2))
    generator.add(Dense(units=784, activation='tanh'))
    return generator
g = create_generator()
g.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 784)               402192    
                                                                 
Total params: 559632 (2.13 MB)
Trainable params: 559632 (2.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(units=512,input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    #discriminator.add(relu(0.2))
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    #discriminator.add(relu(0.2))
    discriminator.add(Dense(units=1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return discriminator

d = create_discriminator()
d.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533505 (2.04 MB)
Trainable params: 533505 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
def create_gan(discriminator, generator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

gan = create_gan(d, g)
gan.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_2 (Sequential)   (None, 784)               559632    
                                                                 
 sequential_3 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 1093137 (4.17 MB)
Trainable params: 559632 (2.13 MB)
Non-trainable params: 533505 (2.04 MB)
_________________________________________________________________


In [18]:
def plot_generated_images(generator):
    noise = np.random.normal(loc=0, scale=1, size=[100, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=(10, 10))
    for i in range(generated_images.shape[0]):
        plt.subplot(10, 10, i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()

In [ ]:
batch_size = 128
epochs = 10000

for e in tqdm(range(epochs)):
    noise = np.random.normal(0,1, [batch_size, 100])
    generated_images=g.predict(noise)
    image_batch=mnist_data[np.random.randint(low=0,high=mnist_data.shape[0],size=batch_size)]
    X = np.concatenate([image_batch, generated_images])
    y_dis = np.zeros(2*batch_size)
    y_dis[:batch_size] = 1
    d.trainable = True
    d.train_on_batch(X, y_dis)

    noise = np.random.normal(0,1, [batch_size, 100])
    y_gen = np.ones(batch_size)
    d.trainable = False
    gan.train_on_batch(noise, y_gen)

    if e == 0 or e % 1000 == 0:
        plot_generated_images(g)

  0%|          | 0/10000 [00:00<?, ?it/s]

4/4 [==============================] - 0s 5ms/step


  0%|          | 1/10000 [00:04<12:33:34,  4.52s/it]

4/4 [==============================] - 0s 5ms/step


  0%|          | 2/10000 [00:04<5:27:36,  1.97s/it] 

...

 47%|████▋     | 4656/10000 [18:02<29:07,  3.06it/s]

4/4 [==============================] - 0s 10ms/step


 47%|████▋     | 4657/10000 [18:02<29:14,  3.05it/s]

4/4 [==============================] - 0s 9ms/step


 47%|████▋     | 4658/10000 [18:03<28:47,  3.09it/s]